In [1]:
import numpy as np

l = [1,2,3,4,5,6,7,8,9,10]
p = 2
r = []
for i in range(p):
    r.append(None)
for i in range(p, len(l)):
    r.append(l[i]*i)
print(r, "LEN:", len(r), "\n\n")

r = []

d = np.diff(l)
for i in range(p):
    r.append(None)
for i in range(p, len(d)):
    r.append(d[i]*i)
print(r, "LEN:", len(r), "\n\n")

[None, None, 6, 12, 20, 30, 42, 56, 72, 90] LEN: 10 


[None, None, 2, 3, 4, 5, 6, 7, 8] LEN: 9 




In [4]:
def calculate_targets(df, period):
    
    targets = []
    all_prices = df

        
    for i in range(0, len(all_prices)-period):
        targets.append(np.sign(all_prices[i+period] - all_prices[i]))
    for i in range(len(all_prices)-period, len(all_prices)):
        targets.append(None)
#     print("TARGETS;PRICES:",len(targets), len(all_prices))
#     df["Target({})".format(period)] = targets
    return targets


prices = [10,11,12,13,14,13,10,9,12,14]

t = calculate_targets(prices, 3)
print(t)
print(len(t)==len(prices))

[1, 1, 1, -1, -1, -1, 1, None, None, None]
True


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

data = pd.read_csv('data.csv')

In [2]:
def RSI(dataframe, period):
    '''
    Computes the RSI of a given price series for a given period length
    :param dataframe:
    :param period:
    :return dataframe with rsi:
    '''

    rsi = []

    for stock in dataframe['Symbol'].unique():
        all_prices = dataframe[dataframe['Symbol'] == stock]['Close']
        diff = np.diff(all_prices) # length is 1 less than the all_prices
        for i in range(period):
            rsi.append(None) # because RSI can't be calculated until period prices have occured

        for i in range(len(diff) - period + 1):
            avg_gain = diff[i:period + i]
            avg_loss = diff[i:period + i]
            avg_gain = abs(sum(avg_gain[avg_gain >= 0]) / period)
            avg_loss = abs(sum(avg_loss[avg_loss < 0]) / period)
            if avg_loss == 0:
                rsi.append(100)
            elif avg_gain == 0:
                rsi.append(0)
            else:
                rs = avg_gain / avg_loss
                rsi.append(100 - (100 / (1 + rs)))

    dataframe['RSI'] = rsi
    return dataframe


def PROC(dataframe, period):
    '''
    Computes the PROC(price rate of change) of a given price series for a given period length
    :param dataframe:
    :param period:
    :return proc:
    '''

    
    proc = []

    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            proc.append(None) # because proc can't be calculated until period prices have occured
        for i in range(len(all_prices) - period):
            if len(all_prices) <= period:
                proc.append(None)
            else:
                proc.append((all_prices[i + period] - all_prices[i]) / all_prices[i])

    dataframe['PROC'] = proc
    return dataframe


def SO(dataframe, period):
    
    so = []
    
    
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        
        for i in range(period):
            so.append(None)
 
        for i in range(len(all_prices) - period):
            C = all_prices[i]
            H = max(all_prices[i:i+period])
            L = min(all_prices[i:i+period])
            so.append(100 * ((C - L) / (H - L)))
                
#     print("Length of so")
#     print(len(so))
#     print("Length of Dataframe")
#     print(len(dataframe))
    dataframe['SO'] = so
    return dataframe

def Williams_R(dataframe, period):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    wr = []
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        for i in range(period):
            wr.append(None) # because proc can't be calculated until period prices have occured
            
        for i in range(period-1,len(all_prices)-1):
            C = all_prices[i]
            H = max(all_prices[i-period+1:i])
            L = min(all_prices[i-period+1:i])
            wr_one = (
                ((H - C) 
                 / (H - L)) * -100
            )
            if wr_one <=-100:
                wr.append(-100)
            elif wr_one >= 100:
                wr.append(100)
            else:
                wr.append(wr_one)
    dataframe["WR"] = wr
    return dataframe



def calculate_targets(df, period):
    
    targets = []

    for stock in df['Symbol'].unique():
        all_prices = list(df[df['Symbol'] == stock]['Close'])
        
        for i in range(0, len(all_prices)-period):
            targets.append(np.sign(all_prices[i+period] - all_prices[i]))
        for i in range(len(all_prices)-period, len(all_prices)):
            targets.append(None)
#     print("TARGETS;PRICES:",len(targets), len(all_prices))
    df["Target({})".format(period)] = targets
    return df


def On_Balance_Volume(dataframe):
    '''
    Williams %R
    Calculates fancy shit for late usage. Nice!

    EXAMPLE USAGE:
    data = pandas.read_csv("./data/ALL.csv", sep=",",header=0,quotechar='"')
    wr = Williams_R(data)
    print(wr)

    '''
    obv = []
    
    for stock in dataframe['Symbol'].unique():
        all_prices = list(dataframe[dataframe['Symbol'] == stock]['Close'])
        all_volumes = list(dataframe[dataframe['Symbol'] == stock]['Volume'])
    
        obv.append(dataframe.iloc[0]["Volume"])
        for i in range(1,len(all_prices)):
            C_old = all_prices[i-1]
            C = all_prices[i]
            if(C > C_old):
                obv.append(obv[i-1]+ all_volumes[i])
            elif (C < C_old):
                obv.append(obv[i - 1] - all_volumes[i])
            else:
                obv.append(obv[i-1])
                
#     print("Length of obv")
#     print(len(obv))
#     print("Length of Dataframe")
#     print(len(dataframe))
    dataframe['OBV'] = obv
    return dataframe

In [3]:
data = RSI(data,14)
print("RSI: Done")

RSI: Done


In [4]:
data = PROC(data, 14)
print("PROC: Done")

C:\Users\Revan\Anaconda3\lib\site-packages\ipykernel\__main__.py:53: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Revan\Anaconda3\lib\site-packages\ipykernel\__main__.py:53: RuntimeWarning: divide by zero encountered in double_scalars


PROC: Done


In [5]:
data = SO(data,14)
print("SO: Done")

C:\Users\Revan\Anaconda3\lib\site-packages\ipykernel\__main__.py:74: RuntimeWarning: invalid value encountered in double_scalars


SO: Done


In [6]:
data = Williams_R(data, 14 )
print("Williams_R: Done")

C:\Users\Revan\Anaconda3\lib\site-packages\ipykernel\__main__.py:106: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Revan\Anaconda3\lib\site-packages\ipykernel\__main__.py:106: RuntimeWarning: invalid value encountered in double_scalars


Williams_R: Done


In [ ]:
data["EWMA"] = pd.ewma(data["Close"], com=.5)
print("EWMA: Done")

C:\Users\Revan\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(adjust=True,min_periods=0,com=0.5,ignore_na=False).mean()
  if __name__ == '__main__':


EWMA: Done


In [ ]:
data = calculate_targets(data, 3)
data = calculate_targets(data, 5)
data = calculate_targets(data, 10)
data = calculate_targets(data, 30)
print('Targets Done - except 60')

In [ ]:
data = calculate_targets(data, 14)

In [ ]:
data

In [ ]:
data.to_csv("./data_preprocessed.csv")

In [ ]:
data = calculate_targets(data, 60)
print("60 Targets: Done")

In [ ]:
data_processed = pd.read_csv("data_preprocessed.csv")
print(data_processed[:10])

In [ ]:
stock= data_processed['Symbol'].unique()[0]
plt.figure(figsize=(10,10))

plt.plot(data_processed[data_processed["Symbol"]==stock]["WR"],label="WR")
plt.plot(data_processed[data_processed["Symbol"]==stock]["SO"],label="So")
plt.plot(data_processed[data_processed["Symbol"]==stock]["RSI"],label="RSI")
plt.plot(data_processed[data_processed["Symbol"]==stock]["PROC"],label="PROC")

plt.plot(data_processed[data_processed["Symbol"]==stock]["Close"],label="Close")
plt.legend()